# Practice Lab M04 (Version 2)
## Data Wrangling and Exploratory Data Analysis

Practice Lab M04 will focus on how to do the data wrangling and exploratory data analysis on big data technologies such as pyspark

## To do

- Use pyspark to perform data wrangling
- Conduct the exploratory data analysis with visulization.


## Tasks 1 Data Wrangling on Pyspark
### Task 1.1 Reading parquet file
We first read the given data source **player.parquet**. However, it is not in a standard readable format. The parquet file is in columnar storage format for efficient storage purpose (very common in cloud service scenario). Our goal in this task is to read this parquet file.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

plt.style.use('bmh')

In [ ]:
!pip install wget
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
# input your code for - read the parquet file by using pyspark 
# print the pyspark dataframe schema
df.show(5)

### Task 1.2 Pyspark operation
After having the dataframe for the given data in pyspark, let's do some wrangling operation by using pyspark

In [ ]:
# filter the data on column BallControl where BallControl > 50

In [ ]:
# filter the data on column  BallControl where BallControl > 50 and BallControl < 70, then print the statistic information of BallControl (using describe())

In [ ]:
# filter the data on column LongPassing where LongPassing > 60 and print the statistic information of BallControl (using describe())

### Task 1.3 Normalization and Standardization
After reading the data and also filter the data, we will sometime need to perform the normalization and standardization on the data.

In [ ]:
# write code for the normalizing the Height on min max scaler and create new column 'scaled_Height'

In [ ]:
# How about we make a custom function to scale columns of our choice
# write code for normalizaing multiple column in the same time 
def min_max_scaler(df, cols_to_scale):
  # Takes a dataframe and list of columns to minmax scale. Returns a dataframe.
  for col in cols_to_scale:
    # Define min and max values and collect them
    max_days = df.agg({col: 'max'}).collect()[0][0]
    min_days = df.agg({col: 'min'}).collect()[0][0]
    new_column_name = 'scaled_' + col
    # Create a new column based off the scaled data
    df = df.withColumn(new_column_name, 
                      (df[col] - min_days) / (max_days - min_days))
  return df

In [ ]:
# try the above function you defined

In [ ]:
# write code for the standardization for the Height and create new column 'standardized_Height', save the dataframe called 'df_stand'



In [ ]:
# Check the mean to be close to 0 


In [ ]:
# And the stddev to be close to 1


### Task 1.4 Pyspark dataframe join
Now we have to join the dataframe. Particularly, we want to join the original dataframe with 'df_stand'.
Firstly, we need to find a ID to join

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

# Using windowfunction to create rownumber on both original dataframe and 'df_stand'
windSpec = Window.orderBy(lit(1))


In [ ]:
# Joining the two dataframe on rownumber and save the final joined dataframe as 'dataframe_pd'



## Task 2 Exploratory Data Analysis
### Task 2.1 Vislauzation for checking distribution 
Now we want to plot the histogram to check the distribution of 'Height' from 'dataframe_pd'

In [ ]:
# df.count() does not include NaN values
df2 = dataframe_pd[[column for column in dataframe_pd if dataframe_pd[column].count() / len(dataframe_pd) >= 0.3]]
del df2['id']
print("List of dropped columns:", end=" ")
for c in dataframe_pd.columns:
    if c not in df2.columns:
        print(c, end=", ")
print('\n')
dataframe_pd = df2

Now lets take a look at how the Height is distributed

In [ ]:
print(dataframe_pd['Height'].describe())


Using pandas dataframe hist() function to plot the distributoins for all the numerical columns
To do so lets first list all the types of our data from our dataset and take only the numerical ones:

In [ ]:
print(list(set(dataframe_pd.dtypes.tolist())))
df_num = dataframe_pd.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8); # ; avoid having the matplotlib verbose informations

### Task 2.2 Boxplot
Now we have to draw the boxplot to check the distribution on discrete columns

In [ ]:
# Discrete the 'LongPassing','Acceleration','SprintSpeed','Agility','Reactions' and create new columns (bin is given in notebook)
columns = ['LongPassing','Acceleration','SprintSpeed','Agility','Reactions']
for col in columns:
  new_column_name = col+'_bin'
  bins = [0, 25, 50, 75, 100]
  labels = [1,2,3,4]
  dataframe_pd[new_column_name] = pd.cut(dataframe_pd[col], bins=bins, labels=labels)

In [ ]:
# Using sns.countplot to plot the boxplot for discrete 'LongPassing'

In [ ]:
# Drawing the sns.countplot with x on discrete 'LongPassing' and y on the 'dribbling' (numerical format)

### Task 2.3 (Advanced) Heatmap visualization
We want to group the discrete 'LongPassing', 'Acceleration' to obtain the count for each. However we want to exclude the 'LongPassing' on bin 1

In [ ]:
# Create the group by with pandas
type_grouped = dataframe_pd[dataframe_pd['LongPassing_bin']>1].groupby(['LongPassing_bin', 'Acceleration_bin']).size()
print(type_grouped)

In [ ]:
# Draw the count into heatmap by using ```sns.heatmap()```